# Tipos/Estructuras en Julia

Vimos que **todo**, en Julia, tiene asociado un tipo o estructura. Aquí veremos varias formas de crear tipos que se acomoden a lo que necesitamos, y algunos trucos para que la ejecución sea rápida. La importancia de los tipos radica, como vimos, en el hecho que la elección del método depende del tipo de sus argumentos.

La convención a la hora de definir tipos es que estén escritos en estilo de "camello", es decir, en que la primer letra de cada palabra empieza en mayúscula. Por ejemplo, tenemos `Float64`, `AbstractFloat`.

Es importante decir que los tipos **no** pueden ser redefinidos o sobreescritos en una sesión de Julia; para hacerlo, hay que iniciar una nueva sesión o reiniciar el Kernel (para el Jupyter notebook).

## Tipos inmutables y constructores internos

In [1]:
struct MiTipo end

Para crear un objeto del tipo `MiTipo` se requiere un *constructor*, que sencillamente es una función que devuelve un objeto del tipo especificado.

In [2]:
methods(MiTipo)

# 1 method for type constructor:
[1] MiTipo() in Main at In[1]:1

La estructura que acabamos de definir **no** contiene ningún tipo de datos, por lo que se llama "singleton". Este tipo de estructuras se utilizan para cuestiones de *dispatch*.

In [3]:
mt = MiTipo()

MiTipo()

In [4]:
typeof(mt)

MiTipo

In [5]:
mt isa MiTipo

true

En general, cuando definimos un tipo nuevo es para que contenga cierto tipo de datos.

In [6]:
struct Partic1d
    x :: Float64
    v :: Float64
end

Por cuestiones de eficiencia es conveniente que los tipos sean concretos; uno puede definir tipos con parámetros, como veremos más adelante. 

Es importante aclarar que las distintas componentes de un tipo pueden tener distintos tipos asociados.

El método que por default crea a un objeto tipo `Partic1d` requiere que especifiquemos *en el mismo orden* todos los *campos* que lo componen.

In [7]:
p1 = Partic1d(1.0, -2.4)

Partic1d(1.0, -2.4)

In [8]:
# p1.<TA>
p1.x

1.0

El tipo de estructura que acabamos de crear es *inmutable*, lo que significa que los campos individuales (cuando son *concretos*), no se pueden cambiar.

In [9]:
isimmutable(p1)

true

In [10]:
p1.x = 2.0

ErrorException: setfield! immutable struct of type Partic1d cannot be changed

La propiedad de inmutabilidad no es recursiva; así, si un objeto consiste de algún campo que es mutable (por ejemplo, `Array{T,N}`, entonces las componentes individuales de ese campo pueden cambiar.

In [11]:
struct Partic2d
    x :: Array{Float64,1}
    v :: Array{Float64,1}
    function Partic2d(x :: Array{Float64,1}, v :: Array{Float64,1})
        @assert length(x) == length(v) == 2
        return new(x, v)
    end
end

La función que aparece en el interior redefine el constructor de default, y se llama *constructor interno*. Hay que notar que *sólo* en este caso se utiliza el comando `new` (ya que el tipo de objeto no existe).

In [12]:
p2 = Partic2d([1.0, 2.5], [1.0, 3.0])

Partic2d([1.0, 2.5], [1.0, 3.0])

No se puede cambiar *el objeto en si* (por ser inmutable), pero sí sus componentes.

In [13]:
p2.x = [2, 1]

ErrorException: setfield! immutable struct of type Partic2d cannot be changed

In [14]:
p2.x[1] = 6.0

6.0

In [15]:
p2.x .= [2, 1]

2-element Array{Float64,1}:
 2.0
 1.0

In [16]:
p2

Partic2d([2.0, 1.0], [1.0, 3.0])

## Tipos mutables

Todo lo dicho anteriormente se puede extender para definir tipos mutables. La única diferencia es que a la hora de definirlos, debemos usar `mutable struct`.

In [17]:
mutable struct MPartic2d
    x :: Array{Float64,1}
    v :: Array{Float64,1}
    function MPartic2d(x :: Array{Float64,1}, v :: Array{Float64,1})
        @assert length(x) == length(v) == 2
        return new(x, v)
    end
end

In [18]:
mp2 = MPartic2d([1.0, 2.5], [1.0, 3.0])

MPartic2d([1.0, 2.5], [1.0, 3.0])

In [19]:
mp2.x = [2, 1]

2-element Array{Int64,1}:
 2
 1

In [20]:
mp2

MPartic2d([2.0, 1.0], [1.0, 3.0])

## Constructores paramétricos

En ocasiones uno quiere definir estructuras que operen con distinto tipo de entradas. Un ejemplo son los vectores: tenemos `Array{Int,1}` y *también* `Array{Float64,1}`, e incluso `Array{Float64 2}`. 

En el ejemplo anterior, `Partic2d` la definimos con campos que son vectores `Array{Float64,1}`, por lo que usar otro tipo de vectores da un error.

In [22]:
Partic2d([1, 2], [1, 3])

MethodError: MethodError: no method matching Partic2d(::Array{Int64,1}, ::Array{Int64,1})

En principio uno *podría* usar en la definición de los campos que componen al tipo, tipos abstractos, como `Real`. Sin embargo, dado que el compilador *no* conoce la estructura en memoria de tipos abstractos, el código será ineficiente. Un ejemplo de código ineficiente, entonces, sería:
    ```julia
    struct Partic3d
        x :: Array{Real,1}
        v :: Array{Real,1}
    end
    ```

La alternativa es definir estructuras *paramétricas*, donde precisamente el parámetro es un tipo concreto (sin especificar) que es subtipo de algún tipo abstracto.

In [23]:
struct Partic3d{T <: Real}
    x :: Array{T,1}
    v :: Array{T,1}
    function Partic3d(x :: Array{T,1}, v :: Array{T,1}) where {T}
        @assert length(x) == length(v) == 3
        return new{T}(x, v)
    end
end

En cierto sentido, en la definición anterior de `Partic3d{T}` la `T` adquiere un tipo concreto que es la que se utiliza en los campos donde se requiere especificar.

In [24]:
Partic3d([1,2,3], [2,3,4])

Partic3d{Int64}([1, 2, 3], [2, 3, 4])

In [25]:
Partic3d([1.5,2,3], [2.5,3,4])

Partic3d{Float64}([1.5, 2.0, 3.0], [2.5, 3.0, 4.0])

In [26]:
Partic3d([1,2,3],[2.5,3,4])

MethodError: MethodError: no method matching Partic3d(::Array{Int64,1}, ::Array{Float64,1})
Closest candidates are:
  Partic3d(::Array{T,1}, !Matched::Array{T,1}) where T at In[23]:5

Dado el árbol de tipos, uno puede lograr a partir del propio diseño del tipo, cierta clase de sobrecarga de operadores, y por lo mismo, la posibilidad de aplicar ciertas funciones a la estructura que hemos creado. Esta propiedad se aplica a todos los tipos que uno define; si uno no especifica el supertipo, éste se considera `Any`.

In [27]:
struct MiVector2d{T <: Real} <: AbstractArray{T,1}
    x :: T
    y :: T
end

In [28]:
x = MiVector2d(1, 2)

MethodError: MethodError: no method matching size(::MiVector2d{Int64})
Closest candidates are:
  size(::AbstractArray{T,N}, !Matched::Any) where {T, N} at abstractarray.jl:38
  size(!Matched::BitArray{1}) at bitarray.jl:77
  size(!Matched::BitArray{1}, !Matched::Integer) at bitarray.jl:81
  ...

El error indica algo *aparentemente* "no relacionado" con lo que hemos hecho, y que tiene que ver con la visualización de `x`. Notemos, por ejemplo, que `x.x` y `x.y` son lo que deben ser! De hecho, `x` ha sido *definido*.

In [29]:
x.x, x.y

(1, 2)

In [30]:
isdefined(Main, :x)

true

Para hacernos la vida más sencilla a la hora de visualizar `MiVector2d`, sobrecargaremos `size` y `getindex`, siguiendo la recomendación de [la documentación](https://docs.julialang.org/en/v1.3/manual/interfaces/#man-interface-array-1).

In [31]:
import Base: size
size(::MiVector2d{T}) where {T} = (2,)

function Base.getindex(v::MiVector2d, i::Int)
    if i == 1
        return v.x
    elseif i == 2
        return v.y
    else
        throw(AssertError)
    end
end

In [32]:
x

2-element MiVector2d{Int64}:
 1
 2

In [33]:
y = MiVector2d(1.2, 2.1)

2-element MiVector2d{Float64}:
 1.2
 2.1

A pesar de que **no** hemos sobrecargado la suma (`:+`), funciona gracias a la estructura de tipo que hemos impuesto a `MiVector2d`.

In [34]:
x + y

2-element Array{Float64,1}:
 2.2
 4.1

Sin embargo, hay que notar que el resultado es un `Array{Float64,1}` y no un `MiVector2d{Float64}`. Para logar que el resultado sea del tipo que queremos, sobrecargamos la función `:+`.

In [35]:
Base.:+(x::MiVector2d, y::MiVector2d) = MiVector2d((x .+ y)...)

In [36]:
x + y

2-element MiVector2d{Float64}:
 2.2
 4.1

Este ejemplo *no* es un ejemplo muy interesante, pero muestra que Julia permite adecuar las cosas a lo que requerimos, y que permite *extender* a Julia para que la interacción sea sencilla.